In [1]:
import robin_stocks as r
from datetime import datetime
from dateutil import tz
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'

from_zone = tz.tzutc()
to_zone = tz.tzlocal()
def utcToLocal(strDate):
    utc = datetime.strptime(strDate, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

import configparser
config = configparser.RawConfigParser()
configFilePath = '/Users/philipmassey/.tokens/robinhood.cfg'
config.read(configFilePath)
rhuser = config.get('login', 'user')
rhpwd = config.get('login', 'pwd')
login = r.login(rhuser,rhpwd)

def diffMaxHighLow(df, strike_price,volume):
    print('Strike\tVolume\tHigh\tLow\tChange\tPercent')
    high_price = max(df.high_price)
    low_price = min(df.high_price)
    diff = round((high_price - low_price),2)
    percent = round(diff/low_price,2)
    print('${:.2f}\t{:,.0f}\t{}\t{}\t{}\t{:,.0%}'.format(strike_price,volume,high_price,low_price,diff,percent))

def figForExpriation(strike_price,volume,optionType,span):
    df = pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span)['data_points'])
    df['strike_price'] = strike_price
    df[['strike_price','high_price','low_price']] = df[['strike_price','high_price','low_price']].apply(pd.to_numeric)
    df = df[df.high_price != df.high_price[0:1][0]]   #filter no changes form the starting date
    print()
    diffMaxHighLow(df,float(strike_price),float(volume))
    local = utcToLocal(df.iloc[-1:].begins_at.values[-1])
    title = '{} Expiration:{}, Time:{:{dfmt} {tfmt}}'.format(symbol,expirationDate,local,dfmt='%Y-%m-%d', tfmt='%H:%M')
    fig = px.line(df, x="begins_at", y="close_price", color="strike_price", line_group="strike_price", hover_name="strike_price",
            line_shape="spline", render_mode="svg")
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Option price",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        ))
    fig.show()

### Expiration Dates

In [5]:
symbol = 'INTC'
expiration_dates = r.get_chains(symbol)['expiration_dates']
print(list(enumerate(expiration_dates)))

[(0, '2020-07-31'), (1, '2020-08-07'), (2, '2020-08-14'), (3, '2020-08-21'), (4, '2020-08-28'), (5, '2020-09-04'), (6, '2020-09-18'), (7, '2020-10-16'), (8, '2021-01-15'), (9, '2021-02-19'), (10, '2021-06-18'), (11, '2022-01-21')]


### Strike Prices for Expiration Date

In [6]:
expirationDate = expiration_dates[4]
volume_limit = 0
optionData = r.find_options_by_expiration([symbol], expirationDate=expirationDate,optionType='call') 
dfoptions = pd.DataFrame((filter(lambda x:x['volume']>volume_limit,optionData)))
#dfoptions[['strike_price','break_even_price','volume','last_trade_price', 'previous_close_price', 'last_trade_size','open_interest']].sort_values(by='strike_price')
#dfoptions[['strike_price','break_even_price','volume','last_trade_price', 'previous_close_price', 'last_trade_size','open_interest']].sort_values(by='volume',ascending=False)
# df1 = dfoptions[['strike_price','volume']][dfoptions.strike_price=='237.5000']
# dfstrike_prices = pd.concat([df1,dfstrike_prices])
dfstrike_prices = dfoptions.sort_values(by='volume',ascending=False)[0:5][['strike_price','volume']].apply(pd.to_numeric)
dfstrike_prices.sort_values(by='strike_price',ascending=True,inplace=True)
dfstrike_prices

*....404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/8ac20ddd-7572-4a16-b5e6-34870f8554e4/
Loading Market Data 404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/0bdfd684-306a-4e65-a5ab-485712e5ed6f/
Loading Market Data 404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/2229221f-d888-4032-b8f3-891a3810d6a3/
Loading Market Data 404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/e3573931-88f0-4235-ba3e-40e7e6348600/
Loading Market Data 404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/b5bc10df-495a-4f97-bddb-2c549c2e5059/
Loading Market Data 404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/952fc7c1-b3e1-4ea6-aad2-c5973bc482c0/
Loading Market Data 404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/881f4df9-6718-4a37-8c3a-4d2b71cb116f/
Loading Market Data 404 Client Erro

TypeError: '>' not supported between instances of 'str' and 'int'

In [58]:
optionType = 'call'
span = 'day'
idx = 3
strike_price = dfstrike_prices.iloc[idx].strike_price
volume = dfstrike_prices.iloc[idx].volume
#strike_price = 40.0
figForExpriation(strike_price,volume,optionType,span)

*....
Strike	Volume	High	Low	Change	Percent
$64.00	29	2.3	0.71	1.59	224%


In [59]:
optionType = 'call'
span = 'week'
figForExpriation(strike_price,volume,optionType,span)


*....
Strike	Volume	High	Low	Change	Percent
$64.00	29	2.3	0.74	1.56	211%


In [60]:
span = 'year'
figForExpriation(strike_price,volume,optionType,span)

*....
Strike	Volume	High	Low	Change	Percent
$64.00	29	2.7	0.01	2.69	26,900%
